In [ ]:

import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import StandardScaler
import glob


In [ ]:

# Define a function to compute binned color features
def bin_spatial(img, color_space='RGB' size=(32,32)):
    
     # convert image to new color space
    if color_space == 'RGB':
            feature_img = img.copy()
    else:
        if color_space == 'HSV':
            feature_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'HLS':
            feature_img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'LUV':
            feature_img = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'YUV':
            feature_img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_img = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
            
    # resize image.
    img_resize = cv2.resize(feature_img, size)
    
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel()
    
    # Return the feature vector
    return features

# Define a function to compute color histogram features
def color_hist(img, nbins=32, bins_range=(0, 256)):
    
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    
    # Return features
    return hist_features

# Define a function to extract features from a list of images
# have this function call bin_spatial() and color_hist()
def extract_features(imgs, cspace='RGB', spatial_size=(32,32),
                        hist_bins=32, hist_range=(0, 256)):
    
    # Create a list to append feature vectors.
    features = []
    
    # Iterate through the list of images
    for img in imgs:
        image = mpimg.imread(img)
        
        # spatial binning
        bin_features = bin_spatial(image, cspace, spatial_size)
        
        # color histogram
        color_features = color_hist(image, hist_bins, hist_range)

        # Append new features
        features.append(np.concatenate((bin_features,color_features)))
        
    # Return features
    return features



In [ ]:

images = glob.glob('*.jpg')
cars = []
notcars = []

for image in images:
    if 'image' in image or 'extra' in image:
        notcars.append(image)
    else:
        cars.append(image)
        
        
car_features = extract_features(cars, cspace='RGB', spatial-size=(32,32),
                               hist_bins=32, hist_range=(0, 256))

notcar_features = extract_features(notcars, cspace='RGB', spatial-size=(32, 32),
                                  bist_bins=32, hist_rnage=(0, 256))

if len(car_features) > 0:
    
    # Create an array stack of features vectors
    X = np.vstack((car_features, notcar_features)).astype(np.float64)   # np.vstack takes float64
    
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    
    # Apply Scaler to X
    scaled_X = X_scaler.transform(X)
    
    car_ind = np.random.randint(0, len(cars))
    
    # Plot an example of raw and scaled features
    fig = plt.figure(figsize=(12,4))
    
    plt.subplot(131)
    plt.imshow(mpimg.imread(cars[car_ind]))
    plt.title('Original Image')
    
    plt.subplot(132)
    plt.plot(X[car_ind])
    plt.title('Raw Features')
    
    plt.subplot(133)
    plt.plot(scaled_X[car_ind])
    plt.title('Normalized Features')
    
    fig.tight_layout()
else:
    print('Your function only return empty feature vectors...')